In [ ]:
### Part I: Knowledge Graph (KG) consturction ###

In [1]:
# Install GLiNER
!pip install gliner

In [2]:
# Define entities
from gliner import GLiNER

def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['label'] == current['label'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['text'] = text[current['start']: next_entity['end']].strip()
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    # Append the last entity
    merged.append(current)
    return merged

# model = GLiNER.from_pretrained("numind/NuNerZero")
model = GLiNER.from_pretrained("numind/NuZero_token")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
# Test entities
labels = ["credit card","loss","theft","misuse"]
labels = [l.lower() for l in labels]

text = """A)	The Cardmember must notify the 24 Hour Call Center immediately if the Primary or any Additional credit card
is misplaced, lost, stolen, mutilated, not received when due or if he/she suspects that the Credit Card is being used
without Cardmember’s permission. Once a card is reported lost, it should not, under any circumstance be used if found
by the Cardmember subsequently. As the Add-On card is an extended facility given on the Primary Card Account,
the Add-on card is rendered invalid when the primary card is reported lost. Similarly, if the Add-on card is reported lost,
the primary Card Account and other Add-on cards are invalidated.
"""

entities = model.predict_entities(text, labels, threshold=0.4)

entities = merge_entities(entities)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Primary => credit card
Additional credit card => credit card
lost => loss
stolen => theft
mutilated => theft
Credit Card => credit card
used => misuse
lost => loss
Add-On card => credit card
Primary Card => credit card
Add-on card => credit card
primary card => credit card
lost => loss
Add-on card => credit card
lost => loss
primary Card => credit card
Add-on => credit card


In [4]:
# Install libraries
!pip install pypdf
!pip install python-dotenv
!pip install pyvis # pyvis creates interactive network visualizations using JavaScript's vis.js. It provides an easy-to-use interface for building and displaying complex graphs in Jupyter notebooks or web apps.

In [5]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-langchain
!pip install llama-index-embeddings-huggingface

In [6]:
# Import tqdm that helps visualize the progress of loops, iterations, and processes in a simple and efficient way.
from tqdm import tqdm

In [7]:
# Install Pandas
!pip install pandas

In [8]:
# Install Python-Docx
!pip install python-docx

In [9]:
from docx import Document

doc = Document('/content/HDFC_subset.docx')  # Update with your filename
for para in doc.paragraphs:
    print(para.text)

Fees and Charges
Fees payable on the Credit Card by the Cardmember


The fees may vary for each Cardmember, and

       
to the Cardmember at the time of applying for the Credit Card. The above fees as applicable are billed to the card account and are stated in the card statement of the month in which it is charged.
Cash Advance Fees
The Cardmember can use the Card to access cash in an emergency from ATMs in India or abroad. A transaction fee of 2.5% (Minimum
` 
the amount withdrawn and would be billed to the Cardmember in the next statement. The transaction fee is subject to change at the discretion of HDFC Bank. All cash advances

       
on revolving credit (please refer schedule of charges) from the date of withdrawal until the

       
subject to change at the discretion of HDFC Bank.
Charges
Charges and fees, as may be applicable from time to time, are payable by Cardmembers for


Cardmember or for defaults committed by the Cardmember with reference to his card account.
HDFC Bank

In [10]:
# Get the full text content by iterating through paragraphs
full_text = "\n".join([para.text for para in doc.paragraphs])

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [12]:
# Now use the full_text with the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    separators=["\n\n", "\n"]
)

chunks = text_splitter.split_text(full_text)
len(chunks)

80

In [13]:
len(chunks[0])

111

In [14]:
labels = ['CreditCard', 'CashFees', 'Charges', 'InterestFree', 'Limits', 'Revocation',
'Loss', 'RewardPoints','Grievance']

In [15]:
chunks_entities = []
entity_list = []
duplicates = set()
for text in tqdm(chunks):
    entities = model.predict_entities(text, labels, threshold=0.7)
    entities = merge_entities(entities)
    chunk_entities = set()
    for entity in entities:
        # print(entity["text"], "=>", entity["label"])
        chunk_entities.add(entity["text"])
        if entity["text"] in duplicates:
            continue
        duplicates.add(entity["text"])
        entity_list.append((entity["text"], "=>", entity["label"]))

    chunks_entities.append(list(chunk_entities))

100%|██████████| 80/80 [03:38<00:00,  2.73s/it]


In [16]:
chunks[30]

'receipt of the cut cards and payment of all amounts outstanding to the card account. No annual, joining or renewal fees shall be refunded on a pro-rata basis.'

In [17]:
entity_list[:20]

[('Fees', '=>', 'Charges'),
 ('Charges\nFees', '=>', 'Charges'),
 ('Credit Card', '=>', 'CreditCard'),
 ('fees', '=>', 'Charges'),
 ('Card', '=>', 'CreditCard'),
 ('transaction fee', '=>', 'Charges'),
 ('revolving credit', '=>', 'CreditCard'),
 ('charges', '=>', 'Charges'),
 ('Charges\nCharges', '=>', 'Charges'),
 ('cancellation charges', '=>', 'Charges'),
 ('Loan on Credit Cards', '=>', 'CreditCard'),
 ('Easy EMI Credit Card', '=>', 'CreditCard'),
 ('Easy EMI', '=>', 'CreditCard'),
 ('20% Rate of Interest', '=>', 'InterestFree'),
 ('Processing Fee', '=>', 'Charges'),
 ('EMI', '=>', 'CreditCard'),
 ('Interest Free', '=>', 'InterestFree'),
 ('interest free', '=>', 'InterestFree'),
 ('Schedule of Charges', '=>', 'Charges'),
 ('HDFC Bank International Platinum Plus Card', '=>', 'CreditCard')]

In [18]:
credit = []
cash = []
charges = []
interest = []
limits = []
revocation = []
loss = []
rewards = []
grievance = []

for e in entity_list:
    s,p, o = e
    if o == 'CreditCard':
        credit.append(s.lower())
    elif o == 'CashFees':
        cash.append(s.lower())
    elif o == 'Charges':
        charges.append(s.lower())
    elif o == 'InterestFree':
        interest.append(s.lower())
    elif o == 'Limits':
        limits.append(s.lower())
    elif o == 'Revocation':
        revocation.append(s.lower())
    elif o == 'Loss':
        loss.append(s.lower())
    elif o == 'RewardPoints':
        rewards.append(s.lower())
    elif o == 'Grievance':
        grievance.append(s.lower())

In [19]:
credit

['credit card',
 'card',
 'revolving credit',
 'loan on credit cards',
 'easy emi credit card',
 'easy emi',
 'emi',
 'hdfc bank international platinum plus card',
 'insta',
 'card',
 'credit card',
 'cardmembers',
 'customers card/account',
 'bank international credit card',
 'cards',
 'primary',
 'additional credit card',
 'credit',
 'add-on card',
 'add-on card',
 'primary',
 'intermiles credit card',
 'silver cards',
 'freedom plus cards',
 'infinia / diners black',
 'easyemi cards',
 'infinia / diners black cards',
 'regalia first credit card',
 'infinia',
 'diners black',
 'regalia',
 'regalia first',
 'credit card',
 'card account']

In [20]:
rewards

['reward points',
 'reward points',
 'reward',
 'intermiles miles',
 'intermiles',
 'cash points',
 'reward points',
 'reward points/cash points',
 '50,000\treward',
 'points',
 'reward/cash points',
 'rps',
 'points',
 'evouchers']

In [21]:
charges

['fees',
 'charges\nfees',
 'fees',
 'transaction fee',
 'charges',
 'charges\ncharges',
 'cancellation charges',
 'processing fee',
 'schedule of charges',
 'related charges',
 'annual',
 'joining',
 'renewal fees']

In [22]:
# Install SPARQLWrapper to simplify querying SPARQL endpoints, which are used to interact with RDF databases and the Semantic Web.
# It acts as a wrapper around SPARQL queries, making it easier to send requests and process responses.
!pip install SPARQLWrapper

In [23]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Define the SPARQL endpoint
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# Define the SPARQL query
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?subject ?predicate ?object
WHERE {
  {
    ?subject ?predicate ?object .
    ?subject rdfs:label "Tom Hanks"@en .
  }
  UNION
  {
    ?subject ?predicate ?object .
    ?subject rdfs:label "Killing Lincoln"@en .
  }
}
LIMIT 100
"""

# Set the query
sparql.setQuery(query)

# Set the return format to JSON
sparql.setReturnFormat(JSON)

# Execute the query and convert the result to a Python dictionary
results = sparql.query().convert()

# Process and print the results
for result in results["results"]["bindings"]:
    print(f"{result['subject']['value']} {result['predicate']['value']} {result['object']['value']}")

http://dbpedia.org/resource/Category:Tom_Hanks http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2004/02/skos/core#Concept
http://dbpedia.org/resource/Tom_Hanks http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing
http://dbpedia.org/resource/Tom_Hanks http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
http://dbpedia.org/resource/Tom_Hanks http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/Person
http://dbpedia.org/resource/Tom_Hanks http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson
http://dbpedia.org/resource/Tom_Hanks http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.wikidata.org/entity/Q19088
http://dbpedia.org/resource/Tom_Hanks http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.wikidata.org/entity/Q215627
http://dbpedia.org/resource/Tom_Hanks http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www

In [26]:
# Install litellm, which acts as a universal API wrapper for multiple Large Language Models (LLMs), including OpenAI, Anthropic, Mistral, Llama, and more.
# It provides a simple interface to interact with different LLM providers using a single API call.
!pip install litellm

In [27]:
from litellm import completion
from typing import List
import json

In [28]:
def format_entities(ent_list:List[str]) -> str:
    return "\n\n".join([e for e in ent_list])

In [ ]:
system_message = """Extract all the relationships between the following entities ONLY based on the given context.
Return a list of JSON objects. For example:

<Examples>
    [{{"subject": "John", "relationship": "lives in", "object": "US"}},
    {{"subject": "Eifel towel", "relationship": "is located in", "object": "Paris"}},
    {{"subject": "Hayao Miyazaki", "relationship": "is", "object": "Japanese animator"}}]
</Examples>

- ONLY return triples and nothing else. None of 'subject', 'relationship' and 'object' can be empty.

Entities: \n\n{entities}

"""
# example for a particular set of entities
i = 3

ents = format_entities(chunks_entities[i])
text = chunks[i]

user_message = "Context: {text}\n\nTriples:"
response = completion(
  api_key="", #Enter your own API Key here
  model="gpt-3.5-turbo",
  messages=[{"content": system_message.format(entities=ents),"role": "system"}, {"content": user_message.format(text=text),"role": "user"}],
  max_tokens=1000,
  format = "json"
)

triples = json.loads(response.choices[0].message.content)
triples

14:09:42 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)


[{'subject': 'transaction fee',
  'relationship': 'is',
  'object': 'subject to change'},
 {'subject': 'transaction fee',
  'relationship': 'is',
  'object': 'billed to the Cardmember'},
 {'subject': 'transaction fee', 'relationship': 'is', 'object': 'withdrawn'},
 {'subject': 'transaction fee',
  'relationship': 'is',
  'object': 'at the discretion of HDFC Bank'}]

14:09:42 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)


In [ ]:
import time

errors = []
all_triples = []
for i in tqdm(range(len(chunks_entities))):
    try:
        ents = format_entities(chunks_entities[i])
        text = chunks[i]

        user_message = "Context: {text}\n\nTriples:"
        response = completion(
             api_key="", #Enter your API Key here
            # model="ollama/adrienbrault/nous-hermes2pro-llama3-8b:q8_0",
            # model="ollama/llama3",
            model="gpt-3.5-turbo",
            messages=[{"content": system_message.format(entities=ents),"role": "system"}, {"content": user_message.format(text=text),"role": "user"}],
            max_tokens=1000,
            format="json"
        )
        triples = json.loads(response.choices[0].message.content)
        all_triples.append(triples)
        time.sleep(3)
    except Exception as e:
        print(f"Error for chunk {i}, {e}")
        errors.append(response.choices[0].message.content)
        all_triples.append([])

  0%|          | 0/80 [00:00<?, ?it/s]14:09:58 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
14:09:58 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
  1%|▏         | 1/80 [00:04<06:10,  4.69s/it]14:10:02 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
14:10:02 - LiteLLM:WARNING: 

Error for chunk 49, Expecting value: line 1 column 1 (char 0)


14:13:47 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
14:13:47 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
 64%|██████▍   | 51/80 [03:53<02:01,  4.19s/it]14:13:51 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
14:13:51 - LiteLLM:WARNING: model_param_helper.py:147 - Error get

Error for chunk 70, Expecting value: line 1 column 1 (char 0)


14:15:15 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
14:15:15 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
 90%|█████████ | 72/80 [05:21<00:31,  3.91s/it]14:15:20 - LiteLLM:WARNING: model_param_helper.py:147 - Error getting transcription kwargs cannot import name 'TranscriptionCreateParamsNonStreaming' from 'openai.types.audio.transcription_create_params' (/usr/local/lib/python3.11/dist-packages/openai/types/audio/transcription_create_params.py)
14:15:20 - LiteLLM:WARNING: model_param_helper.py:147 - Error get

In [31]:
output_file = "triples.json"
json_data = json.dumps(all_triples, indent=4)
with open(output_file, "w") as file:
  file.write(json_data)

In [32]:
input_file = "triples.json"
with open(input_file, "r") as file:
    all_triples = json.load(file)

all_triples[0]

[{'subject': 'Fees', 'relationship': 'payable on', 'object': 'Credit Card'},
 {'subject': 'Fees',
  'relationship': 'may vary for',
  'object': 'each Cardmember'}]

In [33]:
def get_color(n: str) -> str:
    type_to_color = {
        "CreditCard": "#6495ED",
        "Rewards": "#3CB371",
        "Charges": "#F4A460",
    }
    if n.lower() in credit:
        return type_to_color["CreditCard"]
    if n.lower() in rewards:
        return type_to_color["Rewards"]
    if n.lower() in charges:
        return type_to_color["Charges"]
    return "red"

def get_size(n: str) -> int:
    type_to_size = {
        "CreditCard": 50,
        "Rewards": 30,
        "Charges": 20
    }
    if n.lower() in credit:
        return type_to_size["CreditCard"]
    if n.lower() in rewards:
        return type_to_size["Rewards"]
    if n.lower() in charges:
        return type_to_size["Charges"]

    return 10

In [34]:
from pyvis.network import Network
import networkx as nx

G = nx.Graph()

for items in all_triples:
    for item in items:
        try:
            node_1 = item["subject"]
            node_2 = item["object"]
            G.add_node(node_1, title=node_1, color=get_color(node_1), size=get_size(node_1), label=node_1)
            G.add_node(node_2, title=node_2, color=get_color(node_2), size=get_size(node_2), label=node_2)
            G.add_edge(node_1, node_2, title=item["relationship"], weight=4)
        except Exception as e:
            print(f"Error in item: {item}")

In [35]:
#nt =  Network(height="750px", width="100%")
nt =  Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

nt.from_nx(G)
nt.toggle_physics(True)
nt.force_atlas_2based(central_gravity=0.015, gravity=-31)
nt.show("graph.html", notebook=False)
# Generate the HTML
html = nt.generate_html()

# # Write the HTML to a file
with open("graph.html", "w") as file:
    file.write(html)

# # Display the graph in a Jupyter Notebook
from IPython.display import IFrame
IFrame("graph.html", width=1000, height=800)

graph.html


In [36]:
all_triples

[[{'subject': 'Fees', 'relationship': 'payable on', 'object': 'Credit Card'},
  {'subject': 'Fees',
   'relationship': 'may vary for',
   'object': 'each Cardmember'}],
 [{'subject': 'Credit Card',
   'relationship': 'is issued to',
   'object': 'Cardmember'},
  {'subject': 'Credit Card',
   'relationship': 'bills',
   'object': 'applicable fees'},
  {'subject': 'applicable fees',
   'relationship': 'are stated in',
   'object': 'card statement'}],
 [{'subject': 'Cardmember', 'relationship': 'can use', 'object': 'Card'},
  {'subject': 'Card', 'relationship': 'to access', 'object': 'cash'},
  {'subject': 'Card',
   'relationship': 'to access',
   'object': 'cash in an emergency'},
  {'subject': 'Card', 'relationship': 'to access', 'object': 'cash from ATMs'},
  {'subject': 'Card',
   'relationship': 'to access',
   'object': 'cash from ATMs in India or abroad'},
  {'subject': 'Card', 'relationship': 'incur', 'object': 'transaction fee'},
  {'subject': 'transaction fee', 'relationship': 

In [37]:
# Openpyxl is for reading, writing, and modifying Excel (.xlsx) files. It is widely used for data analysis, automation,
# and reporting without requiring Microsoft Excel.
!pip install openpyxl

In [38]:
# Converting JSON file to Excel file, which will be stored in the vector database on Google Cloud
import pandas as pd
import json
import csv

# Assuming all_triples contains the desired data
data = all_triples

# Check if all_triples is empty or if any sublist is empty
if not data or not any(sublist for sublist in data):
    print("Error: 'all_triples' or its sublists are empty. Cannot proceed with CSV writing.")
else:
    # Proceed with writing to CSV if data is available
    with open("triples.csv", mode="w", newline="") as file:
        # Find the first non-empty sublist to get fieldnames
        for sublist in data:
            if sublist:  # Check if the sublist is not empty
                fieldnames = sublist[0].keys()
                break  # Exit the loop after finding the first non-empty sublist
        else:
            # If all sublists are empty, set fieldnames to a default value
            fieldnames = ["subject", "relationship", "object"]  # Or any other suitable default
            print("Warning: All sublists in 'all_triples' are empty. Using default fieldnames.")

        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        for items in data:
            for row in items:  # Iterate through items in each sublist
                writer.writerow(row)

In [ ]:
### Part II: Retrieval Augmented Generation (RAG) to SQL construction ###

In [39]:
# Installing the Required Libraries
!pip install langchain
!pip install chromadb
!pip install google-cloud-bigquery[pandas]
!pip install google-cloud-aiplatform

In [40]:
# Configuring Google Cloud Project and Credentials
VERTEX_PROJECT = "financial-rag-to-sql" # @param{type: "string"}
VERTEX_REGION = "us-central1" # @param{type: "string"}

BIGQUERY_DATASETS = ["financial-rag-to-sql.Bank"] # @param{type: "string"}
BIGQUERY_PROJECT = ["financial-rag-to-sql"] # @param{type: "string"}

In [41]:
from google.colab import auth
auth.authenticate_user()

# Set the project ID
!gcloud config set project financial-rag-to-sql

# Initialize VertexAI with the correct project ID
import vertexai
vertexai.init(project="financial-rag-to-sql", location="us-central1")

Updated property [core/project].


In [42]:
from google.colab import auth
from google.cloud import bigquery

# Authenticate
auth.authenticate_user()

In [43]:
# Building a Vector Database for Table Schema Storage
# Connecting to Big Query using BQ client in python and fetching schema of tables.
import json

#Fetching Schemas of Tables

bq_client = bigquery.Client(project=VERTEX_PROJECT)

# Iterate through each dataset in BIGQUERY_DATASETS
for dataset_id in BIGQUERY_DATASETS: # Use the original value of BIGQUERY_DATASETS
    # Extract the dataset name from the dataset_id string, replacing spaces with underscores
    project_name, dataset_name = dataset_id.split('.')
    #dataset_name = dataset_id.split('.')[-1].replace(' ', '_') # Use dataset_name instead of BIGQUERY_DATASETS

    dataset_ref = bigquery.DatasetReference(project_name, dataset_name)  # Use the dataset reference here

    bq_tables = bq_client.list_tables(dataset_ref) # Use dataset_name here

    schemas = []
    for bq_table in bq_tables:
        # **Change**: Access the first element of BIGQUERY_PROJECT to get the project ID as a string
        t = bq_client.get_table(f"{BIGQUERY_PROJECT[0]}.{dataset_name}.{bq_table.table_id}")
    # **Change**: Create a dictionary containing schema information
    schema_data = {
        "table_name": bq_table.table_id,
        "columns": [{"name": f.name, "type": f.field_type} for f in t.schema]
    }
    # **Change**: Append the structured schema to the list
    schemas.append(json.dumps(schema_data))

print(f"Found {len(schemas)} tables in dataset {BIGQUERY_PROJECT[0]}:{dataset_name}")#import csv # Use dataset_name here

Found 1 tables in dataset financial-rag-to-sql:Bank


In [44]:
# Install the langchain-community package
!pip install -U langchain-community

In [45]:
# Install the required package
!pip install langchain-google-vertexai

In [46]:
# Installing pydantic ensuring data integrity and type safety when dealing with structured data.
!pip install --upgrade langchain google-cloud-aiplatform "pydantic"

In [47]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import VertexAI

embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@003", project=VERTEX_PROJECT, location=VERTEX_REGION)
try: # Avoid duplicated documents
  vector_store.delete_collection()
except:
  print("No need to clean the vector store")
vector_store = Chroma.from_texts(schemas, embedding=embeddings,persist_directory='./data')
n_docs = len(vector_store.get()['ids'])
retriever = vector_store.as_retriever(search_kwargs={'k': 2})
print(f"The vector store has {n_docs} documents")

No need to clean the vector store
The vector store has 1 documents


In [48]:
# Instantiating LLM Models for SQL Query, Interpretation, and Agent Chains
# 3 LLM models for the 3 different chains are instantiated:
# First model is Query Model, which is responsible for generating SQL query based on input and table schema retrieved from vector db similar to input.
# For this “gemini-2.0-flash-001” model is used.
# The other 2 models are default LLM models in ChatVertexAI, which is “gemini-1.5-flash-001”.

from langchain_google_vertexai import ChatVertexAI
from langchain.llms import VertexAI

ChatVertexAI.model_rebuild()
#query_model = ChatVertexAI(model_name="codechat-bison@001", project="northern-symbol-452916-m0", location=VERTEX_REGION, max_output_tokens=1000)
query_model = ChatVertexAI(model_name="gemini-2.0-flash-001", project=VERTEX_PROJECT, location=VERTEX_REGION, max_output_tokens=1000)
interpret_data_model = ChatVertexAI(max_output_tokens=1000, model_name="gemini-2.0-flash-001")
agent_model = ChatVertexAI(max_output_tokens=1024, model_name="gemini-2.0-flash-001")

In [49]:
# SQL prompt used to generate the SQL query for the input.

SQL_PROMPT = """You are a SQL and BigQuery expert.

Your job is to create a query for BigQuery in SQL.

The following JSON object contains the schema of the table used for a query. It has the following format:

{context}

Create a BigQuery SQL query for the following user input, using the above table.
And Use only columns mentioned in schema for the SQL query

The user and the agent have done this conversation so far:
{chat_history}

Follow these restrictions strictly:
- Only return the SQL code.
- Do not add backticks or any markup. Only write the query as output. NOTHING ELSE.
- In FROM, always use the full table path, using `{project}` as project and `{dataset}` as dataset.
- Always transform country names to full uppercase. For instance, if the country is Japan, you should use JAPAN in the query.

User input: {question}

SQL query:
"""

In [50]:
# Define a function which will retrieve relevant documents i.e schemas for the input.

from langchain.schema.vectorstore import VectorStoreRetriever
def get_documents(retriever: VectorStoreRetriever, question: str) -> str:
  # Return only the first document
  output = ""
  for d in retriever.get_relevant_documents(question):
    output += d.page_content
    output += "\n"
    return output

In [51]:
# Define the LLM chain using Langchain expression language syntax.
# Note prompt is defined with 5 placeholder variables and
# later a partial prompt is defined by filling in the 2 placeholder variables project
# and dataset.
# The rest of the variables will get populated with incoming request dictionary consisting of
# input, chat history, and the context variable is populated form the function defined
# above get_documents.

from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

prompt_template = PromptTemplate(
    input_variables=["context", "chat_history", "question", "project", "dataset"],
    template=SQL_PROMPT)

# **Change**: Extract only the dataset name from BIGQUERY_DATASETS
dataset_name = BIGQUERY_DATASETS[0].split('.')[-1]

partial_prompt = prompt_template.partial(project=BIGQUERY_PROJECT,
                                         dataset=dataset_name) # **Change**: Use dataset_name here

# Input will be like {"input": "SOME_QUESTION", "chat_history": "HISTORY"}
docs = {"context": lambda x: get_documents(retriever, x['input'])}
question = {"question": itemgetter("input")}
chat_history = {"chat_history": itemgetter("chat_history")}
query_chain = docs | question | chat_history | partial_prompt | query_model
query = query_chain | StrOutputParser()


In [52]:
# Test the chain using CallBack Handler of Langchain which will show each step of chain execution in detail.
from langchain.callbacks.tracers import ConsoleCallbackHandler
# Example
x = {"input": "Bank's responsibility for checks", "chat_history": ""}
print(query.invoke(x, config={'callbacks': [ConsoleCallbackHandler()]}))

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Bank's responsibility for checks",
  "chat_history": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question,chat_history>] Entering Chain run with input:
{
  "input": "Bank's responsibility for checks",
  "chat_history": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question,chat_history> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": "Bank's responsibility for checks",
  "chat_history": ""
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question,chat_history> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": "Bank's responsibility for checks",
  "chat_history": ""
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question,chat_history> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "Bank's responsibility for checks"
}
[chain/start] [ch

<ipython-input-50-17d072034cb2>:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  for d in retriever.get_relevant_documents(question):


[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question,chat_history> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "{\"table_name\": \"HDFC\", \"columns\": [{\"name\": \"string_field_0\", \"type\": \"STRING\"}, {\"name\": \"string_field_1\", \"type\": \"STRING\"}, {\"name\": \"string_field_2\", \"type\": \"STRING\"}]}\n"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question,chat_history>] s] Exiting Chain run with output:
{
  "context": "{\"table_name\": \"HDFC\", \"columns\": [{\"name\": \"string_field_0\", \"type\": \"STRING\"}, {\"name\": \"string_field_1\", \"type\": \"STRING\"}, {\"name\": \"string_field_2\", \"type\": \"STRING\"}]}\n",
  "question": "Bank's responsibility for checks",
  "chat_history": ""
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "context": "{\"table_name\": \"HDFC\", \"columns\": [{\"name\": \"string_field_0\", \"type\": \"STRING\

In [53]:
# Refine the SQL Chain Output for Interpretation
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.schema.runnable import RunnableLambda

#Refine the Chain output to include other variables in output in dictionary format
def _dict_to_json(x: dict) -> str:
  return "```\n" + json.dumps(x) + "\n```"

query_response_schema = [
    ResponseSchema(name="query", description="SQL query to solve the user question."),
    ResponseSchema(name="question", description="Question asked by the user."),
    ResponseSchema(name="context", description="Documents retrieved from the vector store.")
  ]
query_output_parser = StructuredOutputParser.from_response_schemas(query_response_schema)
query_output_json = docs | question | {"query": query} | RunnableLambda(_dict_to_json) | StrOutputParser()
query_output = query_output_json | query_output_parser

In [54]:
# Example
x = {"input": "A transaction fee of 2.5% (Minimum $50) (excluding Infinia) would be levied on the amount withdrawn and would be billed to the Card member in the next statement.", "chat_history": ""}
output = query_output.invoke(x)  # Output is now a dictionary, input for the next chain

In [55]:
# Building the Interpret Chain for Query Results
INTERPRET_PROMPT = """You are a BigQuery expert. You are also expert in extracting data from CSV.

The following paragraph describes the schema of the table used for a query. It is encoded in JSON format.

{context}

A user asked this question:
{question}

To find the answer, the following SQL query was run in BigQuery:
```
{query}
```

The result of that query was the following table in CSV format:
```
{result}
```

Based on those results, provide a brief answer to the user question.

Follow these restrictions strictly:
- Do not add any explanation about how the answer is obtained, just write the answer.
- Extract any value related to the answer only from the result of the query. Do not use any other data source.
- Just write the answer, omit the question from your answer, this is a chat, just provide the answer.
- If you cannot find the answer in the result, do not make up any data, just say "I cannot find the answer"
"""

In [56]:
# ipython-input-85-5ddc3857d1c0
from google.cloud import bigquery
import google.api_core  # Import the necessary module

def clean_query(query: str):
    # Remove potential leading/trailing spaces and 'sql' keyword
    query = query.strip().replace("sql", "").strip()
    # Remove leading/trailing backticks and unnecessary characters
    cleaned_query = query.strip('`').replace(" ,", ",").replace(",,", ",")
    return cleaned_query


def get_bq_csv(bq_client: bigquery.Client, query: str) -> str:
    cleaned_query = clean_query(query)  # Call clean_query to pre-process the query

    # Extract dataset and table from query
    try:
        from_clause = query.split("FROM")[1].strip()
        parts = from_clause.split(".")
        dataset_name = parts[0].strip().strip("`").strip()
        table_name = parts[1].strip().strip("`").strip()

        # Check if table_name is 'Triples' and if so, replace it with 'Triples_raw_schema'
        # or a correct table name
        if table_name == 'Triples':
            table_name = 'Triples_raw_schema'  # or some other specific table name or table id
            print(f"Table name changed to: {table_name}")  # Print the changed table name

    except IndexError:
        print("Error: Unable to extract dataset and table names from the query.")
        return ""

    if not cleaned_query or cleaned_query.isspace():
        print("Warning: Empty or whitespace-only query generated.")
        return ""

    print("Generated SQL Query:", cleaned_query)

    # Construct the full table path using the project ID, dataset name, and table name
    full_table_path = f"{BIGQUERY_PROJECT[0]}.{dataset_name}.{table_name}"

    # Execute the query
    try:
        df = bq_client.query(cleaned_query, location="us-central1").to_dataframe()
        return df.to_csv(index=False)
    except google.api_core.exceptions.Forbidden as e: # Now 'google' is defined
        print(f"Error: Access denied to table '{full_table_path}'. Please check permissions.")
        print(f"Original error message: {e}")
        return ""

In [57]:
# Define two functions, one is clean_query – this will clean the SQL query of apostrophes and other unnecessary characters
# and the other is get_bq_csv –  this will run the cleaned SQL query in Big Query and get the output table in CSV format.
# Get the output of the previous chain

query = {"query": itemgetter("query")}
context = {"context": itemgetter("context")}
question = {"question": itemgetter("question")}
#cleaned_query = {"result": lambda x: clean_query(x["query"])}
query_result = {"result": lambda x: get_bq_csv(bq_client, x["query"])}

prompt = PromptTemplate(
    input_variables=["question", "query", "result", "context"],
    template=INTERPRET_PROMPT)

run_bq_chain = context | question | query | query_result | prompt
run_bq_result = run_bq_chain | interpret_data_model | StrOutputParser()

In [73]:
# Example
x = {"input": "A transaction fee of 2.5% (Minimum $50) (excluding Infinia) would be levied on the"+
"amount withdrawn and would be billed to the Card member in the next statement.","chat_history": ""}
print("Output of query_output.invoke(x):", query_output.invoke(x))
final_response = run_bq_result.invoke(query_output.invoke(x))

Output of query_output.invoke(x): {'context': '{"table_name": "HDFC", "columns": [{"name": "string_field_0", "type": "STRING"}, {"name": "string_field_1", "type": "STRING"}, {"name": "string_field_2", "type": "STRING"}]}\n', 'question': 'A transaction fee of 2.5% (Minimum $50) (excluding Infinia) would be levied on theamount withdrawn and would be billed to the Card member in the next statement.', 'query': "```sql\nSELECT string_field_0, string_field_1, string_field_2 FROM `financial-rag-to-sql.Bank.HDFC` WHERE string_field_2 LIKE '%transaction fee%' AND string_field_2 LIKE '%2.5%' AND string_field_2 LIKE '%Minimum $50%' AND string_field_2 LIKE '%excluding Infinia%';\n```"}
Generated SQL Query: 
SELECT string_field_0, string_field_1, string_field_2 FROM `financial-rag-to-.Bank.HDFC` WHERE UPPER(string_field_2) LIKE '%TRANSACTION FEE%' AND UPPER(string_field_2) LIKE '%2.5%' AND UPPER(string_field_2) LIKE '%$50%' AND UPPER(string_field_2) NOT LIKE '%INFINIA%';

Error: Access denied to ta

In [81]:
# Implement the Agent Chain for Dynamic Query Routing
# Build the final chain - the Agent Chain. When there is an input, it decides whether to utilise the SQL query tool or to answer it directly.
# Basically, it sends user queries to various tools according to the work that must be completed in order to answer the input.
# Define an agent_memory, agent prompt, and tool funtion.

from langchain.memory import ConversationBufferWindowMemory

agent_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=10,
    return_messages=True)

In [82]:
AGENT_PROMPT = """You are a very powerful assistant that can answer questions using BigQuery.

You can invoke the tool user_question_tool to answer questions using BigQuery.

Always use the tools to try to answer the questions. Use the chat history for context. Never try to use any other external information.

Assume that the user may write with misspellings, fix the spelling of the user before passing the question to any tool.

Don't mention what tool you have used in your answer.
"""

In [83]:
from langchain.tools import tool
from langchain.callbacks.tracers import ConsoleCallbackHandler

@tool
def user_question_tool(question) -> str:
  """Useful to answer natural language questions from users using BigQuery."""
  config={'callbacks': [ConsoleCallbackHandler()]}
  config = {}
  memory = agent_memory.buffer_as_str.strip()
  question = {"input": question, "chat_history": memory}
  query = query_output.invoke(question, config=config)
  print("\n\n******************\n\n")
  print(query['query'])
  print("\n\n******************\n\n")
  result = run_bq_result.invoke(query, config=config)
  return result.strip()

In [84]:
# Bring together all the main components of agent and initialize the agent.
from langchain.agents import AgentType, initialize_agent, AgentExecutor

agent_kwargs = {"system_message": AGENT_PROMPT}
agent_tools = [user_question_tool]

agent_memory.clear()

agent = initialize_agent(
    tools=agent_tools,
    llm=agent_model,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=agent_memory,
    agent_kwargs=agent_kwargs, # Fixed the typo here from agent_kwgards to agent_kwargs
    max_iterations=5,
    early_stopping_method='generate',
    verbose=True)

In [85]:
q = "A transaction fee of 2.5% (Minimum $50) (excluding Infinia) would be levied on the amount withdrawn and would be billed to the Card member in the next statement."
agent.invoke(q)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "This describes a transaction fee structure. A 2.5% fee (with a minimum of $50) will be charged on withdrawals, excluding Infinia cardholders. This fee will appear on the card member's next statement."
}
```

> Finished chain.


{'input': 'A transaction fee of 2.5% (Minimum $50) (excluding Infinia) would be levied on the amount withdrawn and would be billed to the Card member in the next statement.',
 'chat_history': [],
 'output': "This describes a transaction fee structure. A 2.5% fee (with a minimum of $50) will be charged on withdrawals, excluding Infinia cardholders. This fee will appear on the card member's next statement."}